In [1]:
import pandas as pd
import requests
from datetime import date, timedelta
from ml_functions import get_rf_ensemble_plot
from skgarden import RandomForestQuantileRegressor
from plotly.io import write_html
import plotly.express as px
import numpy as np




C:\Users\willi\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\willi\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\willi\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be remo

In [ ]:
end_date = date.today().isoformat()

In [ ]:
end_date

In [ ]:
delta = timedelta(730)

In [ ]:
start_date = (date.today() - delta).isoformat()

In [ ]:
start_date

In [ ]:
url = 'https://api.exchangeratesapi.io/history'


In [ ]:
payload = {
    "start_at": start_date,
    "end_at": end_date,
    "symbols": "CNY,JPY,EUR",
    "base": "USD"
}

In [ ]:
result = requests.get(url, params = payload).json()

In [ ]:
rate_df = pd.DataFrame(result['rates']).transpose()

In [ ]:
rate_df = rate_df.sort_index()


In [ ]:
rate_df.index = pd.to_datetime(rate_df.index, format = '%Y-%m-%d')

In [ ]:
rate_df

In [ ]:
btc_price = get_historical_data('BTC', 730)

In [ ]:
len(btc_price)

In [ ]:
btc_price = btc_price.rename(columns = {'close': 'BTC'})
    
    

In [ ]:
btc_price.head()

In [ ]:
eth_price = get_historical_data('ETH', 730)

In [ ]:
eth_price = eth_price.rename(columns = {'close': 'ETH'})

In [ ]:
ltc_price = get_historical_data('LTC', 730)

In [ ]:
ltc_price = ltc_price.rename(columns = {'close': 'LTC'})

In [ ]:
coin_df = btc_price

In [ ]:
coin_df['LTC'] = ltc_price

In [ ]:
coin_df['ETH'] = eth_price

In [ ]:
combined_df = pd.concat([coin_df, rate_df], axis = 1 )

In [ ]:
combined_df.head()

In [ ]:
combined_df['EUR'] = combined_df['EUR'].ffill()


In [ ]:
combined_df.head()

In [ ]:
combined_df['CNY'] = combined_df['JPY'].ffill()


In [ ]:
combined_df['JPY'] = combined_df['CNY'].ffill()

In [ ]:
combined_df.tail()

In [ ]:
combined_df.head()

In [ ]:
combined_df = combined_df.reset_index()

In [ ]:
combined_df.head()

In [ ]:
delta = timedelta(1)
add_row_date = (date.today() + delta).isoformat()

In [ ]:
df2 = pd.DataFrame([[add_row_date,0,0,0,0,0,0]], columns= ['index','BTC','LTC','ETH','EUR','CNY','JPY'])

In [ ]:
combined_df = combined_df.append(df2, ignore_index = True)

In [ ]:
combined_df = combined_df.set_index('index')

In [ ]:
combined_df.tail()

In [ ]:
combined_df.tail()

In [ ]:
combined_df.tail()

In [ ]:
combined_df['BTC_Previous_Day'] = combined_df['BTC'].shift(1)

In [ ]:
combined_df.tail()

In [ ]:
combined_df['LTC_Previous_Day'] = combined_df['LTC'].shift(1)

In [ ]:
combined_df['ETH_Previous_Day'] = combined_df['ETH'].shift(1)

In [ ]:
combined_df['EUR_Previous_Day'] = combined_df['EUR'].shift(1)

In [ ]:
combined_df['CNY_Previous_Day'] = combined_df['CNY'].shift(1)

In [ ]:
combined_df['JPY_Previous_Day'] = combined_df['JPY'].shift(1)

In [ ]:
combined_df.tail()

In [ ]:
combined_df = combined_df.drop(columns = ['EUR', 'CNY', 'JPY'])

In [ ]:
combined_df = combined_df.dropna()

In [ ]:
combined_df.tail()

In [ ]:
new_df = combined_df.copy()

In [ ]:
new_df = get_random_forest_df()

In [ ]:
new_df

In [ ]:
len(new_df)


In [ ]:
X = new_df.drop(columns = ['BTC', 'LTC', 'ETH'])


In [ ]:
X.tail()

In [ ]:
y = new_df['BTC']

In [ ]:
X_train = X[:725]


In [ ]:
y_train = y[:725]

In [ ]:
X_test = X[725:]

In [ ]:
y_test = y[725:]

In [ ]:
regressor = RandomForestQuantileRegressor(random_state = 0, n_estimators = 500)

In [ ]:
model = regressor.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
print(model.feature_importances_)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
actuals = pd.DataFrame(y_test)

In [ ]:
actuals_vs_predictions = actuals.copy()

In [ ]:
actuals_vs_predictions['Predictions'] = predictions

In [ ]:
actuals_vs_predictions = actuals_vs_predictions.reset_index()

In [ ]:
actuals_vs_predictions

In [ ]:
fig = px.line(actuals_vs_predictions, x = 'index', y = 'Predictions')
fig.update_layout(autosize = True, height = 800, width = 950, title_text = 'BTC Random Forest Ensemble', template = 'seaborn')
fig.update_xaxes(nticks = 5, title = 'Date')
fig.update_yaxes(automargin=True)

In [2]:
get_rf_ensemble_plot()